In [ ]:
import numpy as np
import scipy
from scipy.optimize import curve_fit


# Load the npz file
data = np.load('/Users/neilhazra/Downloads/pos25mT_553_50mW.npz')['data']

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
def find_peaks_split(frame, params=[]):
    x, y= np.meshgrid(np.arange(-frame.shape[0]//2, +frame.shape[0]//2), np.arange(-frame.shape[0]//2, +frame.shape[0]//2))
    r = x**2 + y**2
    theta = np.atan2(x,y)

    thetas = np.linspace(-np.pi,np.pi, 360)
    dtheta = thetas[1] - thetas[0]
    theta_slices = []
    for i, th in enumerate(thetas):
        theta_slices.append(frame[(theta < (th + dtheta/2)) & (theta >(th - dtheta/2))].mean())
    theta_intensity = np.nan_to_num(np.array(theta_slices))
    n = 5
    convolved = scipy.ndimage.convolve(theta_intensity, [1/n]*n, mode='wrap')
    def grid_search_curve_fit(func, xdata, ydata, param_grid, prev_estimate, bounds=None):
        param_values = list(param_grid.values())
        
        # Generate all combinations of initial parameters
        from itertools import product
        param_combinations = list(product(*param_values))
        
        # Initialize variables to store the best fit
        best_params = None
        best_covariance = None
        best_error = np.inf
        
        # Loop over all parameter combinations
        for initial_guess in param_combinations + prev_estimate:
            try:
                # Fit using the current set of initial parameters
                params, covariance = curve_fit(func, xdata, ydata, p0=initial_guess)
                
                # Calculate the residual sum of squares (RSS)
                residuals = ydata - func(xdata, *params)
                rss = np.sum(residuals**2)
                
                # Check if this fit is better than the previous best
                if rss < best_error:
                    best_error = rss
                    best_params = params
                    best_covariance = covariance
            
            except RuntimeError:
                print('not convergred')
                # Catch the case where curve_fit fails to converge for certain parameter sets
                pass
        return best_params, best_covariance, best_error

    # Define the function for a mixture of Gaussian peaks distributed at angles in degrees
    def gaussian_mixture(x, w1, sigma1, theta1_start, w2, theta2_start):
        # Convert angles to radians for computation (since Gaussians use squared distance)
        x_rad = np.radians(x)
        
        # First set of 6 peaks (60 degrees apart)
        peaks1 = [(theta1_start + i * 60) % 360 for i in range(6)]
        
        # Second set of 6 peaks (60 degrees apart)
        peaks2 = [(theta2_start + i * 60) % 360 for i in range(6)]
        
        # Convert peak angles to radians
        peaks1_rad = np.radians(peaks1)
        peaks2_rad = np.radians(peaks2)
        
        # Gaussian contributions from the first set of peaks
        gaussian1 = np.zeros_like(x_rad)
        for peak in peaks1_rad:
            gaussian1 += w1 * np.exp(-0.5 * ((x_rad - peak) / sigma1)**2) / np.sqrt(2 * np.pi * sigma1**2)
        
        # Gaussian contributions from the second set of peaks
        gaussian2 = np.zeros_like(x_rad)
        for peak in peaks2_rad:
            gaussian2 += w2 * np.exp(-0.5 * ((x_rad - peak) / sigma1)**2) / np.sqrt(2 * np.pi * sigma1**2)
        
        # Total mixture of both sets of peaks
        return gaussian1 + gaussian2 + np.mean(convolved) * 2/3

    param_grid = {
        'w1': [0.25, 0.5, 1],   # Variations around 40
        'sigma1': [0.008, 0.01, 0.02],  # Variations around 1
        'mu1': [np.argmax(convolved)],  # Variations around 20
        'w2': [0.25, 0.5, 1],   # Variations around 20
        'mu2': [np.argmax(convolved) - 15, np.argmax(convolved) - 10, np.argmax(convolved) + 10, np.argmax(convolved) + 15],  # Variations around 50
    }

    params, _, _ = grid_search_curve_fit(gaussian_mixture, np.arange(convolved.shape[0]), convolved, param_grid=param_grid, prev_estimate=params)
    print(params)
    w1_fit, sigma1_fit,mu1_fit, w2_fit, mu2_fit = params
    fitted_pdf = gaussian_mixture(np.arange(convolved.shape[0]), w1_fit, sigma1_fit, mu1_fit, w2_fit, mu2_fit)
    return mu1_fit % 60, mu2_fit % 60, fitted_pdf, convolved, params

In [ ]:
frame = data[250]
frame = data[225]
frame = data[109]
plt.imshow(frame)

In [ ]:
theta_1, theta_2, fitted_pdf, theta_intensity, params = find_peaks_split(data[275])
print(theta_1, theta_2)
plt.plot(theta_intensity)
plt.plot(fitted_pdf)
plt.show()

In [ ]:
theta_1s =  []
theta_2s = []
prev = None
frames = []
for i, frame in enumerate(data):
    if prev is None:   
        theta_1, theta_2, _ , _, prev = find_peaks_split(frame, [prev])
    else:
        theta_1, theta_2, _ , _, prev = find_peaks_split(frame)
    theta_1s.append(theta_1)
    theta_2s.append(theta_2)
    frames.append(i)
    print(i, 'out of', len(data))

In [ ]:
from scipy.ndimage import median_filter

plt.plot(frames, median_filter(theta_1s, 25), label='theta1')
plt.plot(frames,  median_filter(theta_2s, 25), label = 'theta2')
plt.xlabel('Frame index')
plt.ylabel('theta_1')
plt.legend()
plt.show()